In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

def load_model(model_path):
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype="auto",
        device_map="auto"
    )
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    return model, tokenizer



C:\Users\User\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def generate_response(model, tokenizer, prompt, system_prompt):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

In [3]:
from sentence_transformers import SentenceTransformer
import numpy as np
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from sentence_transformers import SentenceTransformer
import json

import json

# Load JSONL data
def load_jsonl(file_path):
    data = []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Skipping invalid line: {line.strip()} - Error: {e}")
                continue  # Skip malformed lines

    return data

rag_jsonl_path = "./training_data/raw_data/base_train.jsonl" 

# Example: JSONL structure
# {"question": "What is RAG?", "answer": "Retrieval-Augmented Generation..."}
jsonl_data = load_jsonl(rag_jsonl_path)


embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


# Convert data into LangChain Document format
documents = [Document(page_content=item["input"], metadata={"output": item["output"]}) for item in jsonl_data]

# Create FAISS vector store
vector_store = FAISS.from_documents(documents, embed_model)

# Save FAISS index
vector_store.save_local("faiss_knowledge_base")



Skipping invalid line:  - Error: Expecting value: line 2 column 1 (char 1)
Skipping invalid line:  - Error: Expecting value: line 2 column 1 (char 1)
Skipping invalid line:  - Error: Expecting value: line 2 column 1 (char 1)
Skipping invalid line:  - Error: Expecting value: line 2 column 1 (char 1)
Skipping invalid line:  - Error: Expecting value: line 2 column 1 (char 1)
Skipping invalid line:  - Error: Expecting value: line 2 column 1 (char 1)
Skipping invalid line:  - Error: Expecting value: line 2 column 1 (char 1)
Skipping invalid line:  - Error: Expecting value: line 2 column 1 (char 1)
Skipping invalid line:  - Error: Expecting value: line 2 column 1 (char 1)
Skipping invalid line:  - Error: Expecting value: line 2 column 1 (char 1)
Skipping invalid line:  - Error: Expecting value: line 2 column 1 (char 1)
Skipping invalid line:  - Error: Expecting value: line 2 column 1 (char 1)
Skipping invalid line:  - Error: Expecting value: line 2 column 1 (char 1)
Skipping invalid line:  -

C:\Users\User\AppData\Local\Temp\ipykernel_9956\2613297458.py:31: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [4]:
# Load the FAISS index later
vector_store = FAISS.load_local("faiss_knowledge_base", embed_model, allow_dangerous_deserialization=True)

In [13]:
def retrieve_docs(query, top_k=3, threshold=0.5):
    docs = vector_store.similarity_search_with_score(query, top_k)

    results = ""

    for doc in docs:
        if doc[1] < threshold:
            results += f" {doc[0].metadata['output']}\n"


    return results if results else "NIL"

    
print(retrieve_docs("Who is the president of the USA?", top_k=3, threshold=1.0))


NIL


In [6]:
import random

def generate_response_w_RAG(model, tokenizer, prompt):

    rejection_messages = [
    "C'mon now. I'm not ChatGPT or Google.",
    "No clue, try somewhere else",
    "I have no idea, go search online.",
    "Hey, I'm not a search engine.",
    "I'm not sure, try a friend.",
    ]

    results = retrieve_docs(prompt)
    if results == "NIL":
        return random.choice(rejection_messages)
    rag_prompt = "The user is asking for information about Wei Hong, you are to respond as him. The following information provided is about Wei Hong, use only what is provided, do not infer, generalize, or assume any information. If no relavant information is provided, respond with 'I don't know':{}".format(results)
    messages = [
        {"role": "system", "content": rag_prompt},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512,
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

def filter_response(response):
    # Implement filtering logic here
    # For example, remove any unwanted characters or phrases
    return response.replace("I don't know", "I'm not sure about that.")

In [11]:
import json
import evaluate

In [ ]:

model_path = "./trained_models/0.5b-v8"
model, tokenizer = load_model(model_path)

[(Document(id='323b147b-af5b-4fbf-ab80-a67215ca5375', metadata={'output': "I'm interested in technology, finance, and AI."}, page_content='What are your interests?'), 0.39242497), (Document(id='24670c43-e707-47ae-ad69-9c294aa36535', metadata={'output': 'Technology, finance, and AI are my main interests—I like optimizing systems, whether in code or real-world strategies.'}, page_content='What fields interest you the most?'), 0.7239202), (Document(id='fa6b131d-9b73-4cd9-89fb-63ea90cbf316', metadata={'output': "I'm looking for roles in software engineering, AI/ML, or systems/business analysis—anything that blends problem-solving with efficiency."}, page_content='What career opportunities are you interested in?'), 0.8049828)]


In [37]:
input_text = "Any new projects?"
model_output = generate_response_w_RAG(model, tokenizer, input_text)
print(model_output)

Retrieved :  I am currently exploring LLMs through fine-tuning and, RAG. Also looking into image Gen models, control nets and LoRA

Yes, I'm working on a language model for text generation, specifically in the realm of domain-specific questions and responses.


In [16]:

def load_eval_data(file_path):
    data = []
    with open(file_path, "r") as f:
        for line in f:
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                continue

    return data

def evaluate_model(model, tokenizer, eval_path):

    eval_data = load_eval_data(eval_path)

    # Load Metrics
    bleu_metric = evaluate.load("sacrebleu")


    # Evaluate Performance
    total_count = len(eval_data)
    exact_match_count = 0
    bleu_scores = []

    test_log = {}

    for example in eval_data:
        input_text = example["input"]
        expected_output = example["output"]

        # Generate model response
        model_output = generate_response_w_RAG(model, tokenizer, input_text)

        # Exact match
        if model_output.strip().lower() == expected_output.strip().lower():
            exact_match_count += 1

        # BLEU score
        bleu_score = bleu_metric.compute(predictions=[model_output], references=[[expected_output]])["score"]
        bleu_scores.append(bleu_score)

        test_log[input_text] = {
            "expected": expected_output,
            "generated": model_output,
            "bleu_score": bleu_score
        }

    # Compute Final Metrics
    accuracy = exact_match_count / total_count
    average_bleu = sum(bleu_scores) / total_count

    # print("\n📈 **Final Evaluation Results:**")
    # print(f"🎯 Accuracy: {accuracy * 100:.2f}%")
    # print(f"📊 Average BLEU Score: {average_bleu:.2f}")
    return test_log, accuracy, average_bleu


In [8]:
model_path_1 = "./trained_models/0.5b-v8"
model_path_2 = "./trained_models/0.5b-v12"
# model_path_3 = "./trained_models/0.5b-v6"
base_model_path = "./base_models/Qwen2.5-0.5B-inst"
EVAL_FILE = "./training_data/raw_data/eval.jsonl"  
SYSTEM_PROMPT = "The user is asking for information about Wei Hong, you are to respond as him. Use only data that you are trained on, do not infer, generalize, or assume any information. If you do not know the answer, respond with 'I don't know'"


In [9]:
import torch
torch.cuda.empty_cache()

model_1, tokenizer_1 = load_model(model_path_1)
model_2, tokenizer_2 = load_model(model_path_2)
# model_3, tokenizer_3 = load_model(model_path_3)
# bmodel, btokenizer = load_model(base_model_path)

In [ ]:

# print("v8  ---------------")
# log1, accuracy1, bleu1 = evaluate_model(model_1, tokenizer_1, EVAL_FILE)
# print("")
# print("v12  ---------------")
# log2, accuracy2, bleu2 = evaluate_model(model_2, tokenizer_2, EVAL_FILE)
# print("")
# print("v6  ---------------")
# log3, accuracy3, bleu3 = evaluate_model(model_3, tokenizer_3, EVAL_FILE)
# print("")
# print("Base model ---------------")
# logb, accuracyb, bleub = evaluate_model(bmodel, btokenizer, EVAL_FILE)

model1 = 0
model2 = 0 
for i in range (5):
    
    log1, accuracy1, bleu1 = evaluate_model(model_1, tokenizer_1, EVAL_FILE)
    log2, accuracy2, bleu2 = evaluate_model(model_2, tokenizer_2, EVAL_FILE)
    model1 += bleu1
    model2 += bleu2

print("v8  ---------------")
print(f"Average BLEU Score: {model1/5:.2f}")
print("")
print("v12  ---------------")
print(f"Average BLEU Score: {model2/5:.2f}")


v8  ---------------

📈 **Final Evaluation Results:**
🎯 Accuracy: 0.00%
📊 Average BLEU Score: 5.61

v12  ---------------

📈 **Final Evaluation Results:**
🎯 Accuracy: 0.00%
📊 Average BLEU Score: 5.38
{'Can you share a bit about your background?': {'expected': "I'm Wei Hong, a computer science major with a strong interest in software development and AI research.", 'generated': "I'm not sure, try a friend.", 'bleu_score': 1.9746392774230568}, 'Whatâ€™s your experience in software development?': {'expected': "I've worked on multiple web applications, backend systems, and machine learning projects, gaining hands-on experience with modern technologies.", 'generated': 'I have experience in both designing beautiful frontends and building scalable backend systems, designing REST APIs, and hosting applications using Docker on GCP.', 'bleu_score': 7.831957705977147}, 'How did you become interested in programming?': {'expected': 'I was fascinated by technology from a young age, curious about how co